In [1]:
from ssd import build_ssd

## box_utils裡面很多Function，可以看看是怎麼設計的
from layers.box_utils import * 
from torch.autograd import Variable
from layers import functions
from layers import modules
from math import sqrt 
from itertools import product
from torch.autograd import Function

import os
import sys
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import argparse
import pickle
import torch.nn.functional as F


In [2]:
## 詳細模型結構可以參考ssd.py
ssd_net=build_ssd('train', size=300, num_classes=21)

C:\Users\Sanga\repos\1st-DL-CVMarathon\Object Detection 程式導讀\ssd.py:34: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  self.priors = Variable(self.priorbox.forward(), volatile=True)


## 默認Config檔案在data/config.py內

In [3]:
ssd_net.cfg

{'num_classes': 21,
 'lr_steps': (80000, 100000, 120000),
 'max_iter': 120000,
 'feature_maps': [38, 19, 10, 5, 3, 1],
 'min_dim': 300,
 'steps': [8, 16, 32, 64, 100, 300],
 'min_sizes': [30, 60, 111, 162, 213, 264],
 'max_sizes': [60, 111, 162, 213, 264, 315],
 'aspect_ratios': [[2], [2, 3], [2, 3], [2, 3], [2], [2]],
 'variance': [0.1, 0.2],
 'clip': True,
 'name': 'VOC'}

In [4]:
cfg = {
    'num_classes': 21,
    'lr_steps': (80000, 100000, 120000),
    'max_iter': 120000,
    'feature_maps': [38, 19, 10, 5, 3, 1],
    'min_dim': 300,
    'steps': [8, 16, 32, 64, 100, 300],
    'min_sizes': [30, 60, 111, 162, 213, 264],
    'max_sizes': [60, 111, 162, 213, 264, 315],
    'aspect_ratios': [[2], [2, 3], [2, 3], [2, 3], [2], [2]],
    'variance': [0.1, 0.2],
    'clip': True,
    'name': 'VOC',
}

### 'aspect_ratios' : 使用六張Feature Map，每一張上方有預設的anchor boxes，Boxes aspect ratio可以自己設定
### 'feature_maps' : 使用feature map大小為[38x38, 19x19, 10x10, 5x5, 3x3, 1x1]
### 'min_sizes'、'max_sizes'可藉由下方算式算出，由作者自行設計
### 'steps' : Feature map回放回原本300*300的比例，如38要回放為300大概就是8倍
### 'variance' : Training 的一個trick，加速收斂，詳見：https://github.com/rykov8/ssd_keras/issues/53

---

## 'min_sizes'、'max_sizes' 計算

In [22]:
import math
## source:https://blog.csdn.net/gbyy42299/article/details/81235891
min_dim = 300   ## 维度
# conv4_3 ==> 38 x 38
# fc7 ==> 19 x 19
# conv6_2 ==> 10 x 10
# conv7_2 ==> 5 x 5
# conv8_2 ==> 3 x 3
# conv9_2 ==> 1 x 1
mbox_source_layers = ['conv4_3', 'fc7', 'conv6_2', 'conv7_2', 'conv8_2', 'conv9_2'] ## prior_box來源層，可以更改。很多改進都是基於此處的調整。
# in percent %
min_ratio = 20 ## 這裡即是論文中所說的Smin的= 0.2，Smax的= 0.9的初始值，經過下面的運算即可得到min_sizes，max_sizes。
max_ratio = 90
step = int(math.floor((max_ratio - min_ratio) / (len(mbox_source_layers) - 2)))## 取一個間距步長，即在下面用於循環給比取值時起一個間距作用。可以用一個具體的數值代替，這裡等於17。
min_sizes = []  ## 經過以下運算得到min_sizes和max_sizes。
max_sizes = []
for ratio in range(min_ratio, max_ratio + 1, step):
    ## 從min_ratio至max_ratio + 1每隔步驟= 17取一個值賦值給比。注意範圍函數的作用。
    ## min_sizes.append（）函數即把括號內部每次得到的值依次給了min_sizes。
    min_sizes.append(min_dim * ratio / 100.)
    max_sizes.append(min_dim * (ratio + step) / 100.)
min_sizes = [min_dim * 10 / 100.] + min_sizes
max_sizes = [min_dim * 20 / 100.] + max_sizes

## steps: 這一步要仔細理解，即計算卷積層產生的prior_box距離原圖的步長，先驗框中心點的坐標會乘以step，
## 相當於從特徵映射位置映射回原圖位置，比如conv4_3輸出特徵圖大小為38 *38，而輸入的圖片為300* 300，
## 所以38 *8約等於300，所以映射步長為8.這是針對300* 300的訓練圖片。
steps = [8, 16, 32, 64, 100, 300]  
aspect_ratios = [[2], [2, 3], [2, 3], [2, 3], [2], [2]]
 
print('min_sizes: ',min_sizes)
print('max_sizes: ',max_sizes)


min_sizes:  [30.0, 60.0, 111.0, 162.0, 213.0, 264.0]
max_sizes:  [60.0, 111.0, 162.0, 213.0, 264.0, 315.0]


---

## Default anchor boxes設計原理，看懂收穫很多
##### 可以理解 SSD原文中 8732個anchors是怎麼來的
##### 38×38×4+19×19×6+10×10×6+5×5×6+3×3×4+1×1×4=8732

In [23]:
class PriorBox(object):
    """Compute priorbox coordinates in center-offset form for each source
    feature map.
    """
    def __init__(self, cfg):
        super(PriorBox, self).__init__()
        self.image_size = cfg['min_dim']
        # number of priors for feature map location (either 4 or 6)
        self.num_priors = len(cfg['aspect_ratios'])
        self.variance = cfg['variance'] or [0.1]
        self.feature_maps = cfg['feature_maps']
        self.min_sizes = cfg['min_sizes']
        self.max_sizes = cfg['max_sizes']
        self.steps = cfg['steps']
        self.aspect_ratios = cfg['aspect_ratios']
        self.clip = cfg['clip']
        self.version = cfg['name']
        for v in self.variance:
            if v <= 0:
                raise ValueError('Variances must be greater than 0')

    def forward(self):
        mean = []
        '''依照Feature map大小找出所有的pixel 中心'''
        '''下方這兩個loop會找出W個x軸pixel對上W個y軸pixel，假如現在是在38x38的feature map上，就會有38x38個值'''
        '''ex. [0,1],[0,2]..[0,37] [1,1],[1,2]..[1,37]..........[37,37]'''
        for k, f in enumerate(self.feature_maps):
            for i, j in product(range(f), repeat=2):
                f_k = self.image_size / self.steps[k] ## 如self.steps==8，就是先將原圖size normalize(/300)後再乘上8
                # unit center x,y
                '''中心點'''
                cx = (j + 0.5) / f_k
                cy = (i + 0.5) / f_k

                # aspect_ratio: 1
                # rel size: min_size
                '''/self.image_size 就是在做normalization '''
                s_k = self.min_sizes[k]/self.image_size
                '''小的正方形box'''
                mean += [cx, cy, s_k, s_k]

                # aspect_ratio: 1
                # rel size: sqrt(s_k * s_(k+1))
                '''大的正方形box'''
                s_k_prime = sqrt(s_k * (self.max_sizes[k]/self.image_size))
                mean += [cx, cy, s_k_prime, s_k_prime]

                # rest of aspect ratios
                for ar in self.aspect_ratios[k]:
                    '''aspect ratio 2,3'''
                    mean += [cx, cy, s_k*sqrt(ar), s_k/sqrt(ar)]
                    '''aspect ratio 1/2,1/3'''
                    mean += [cx, cy, s_k/sqrt(ar), s_k*sqrt(ar)]
        # back to torch land
        output = torch.Tensor(mean).view(-1, 4)
        if self.clip:
            output.clamp_(max=1, min=0)
        return output

In [24]:
PriorBox_Demo=PriorBox(cfg)

In [25]:
print(PriorBox_Demo.forward().shape)

torch.Size([8732, 4])


## Loss 如何設計

In [26]:
class MultiBoxLoss(nn.Module):

    def __init__(self, num_classes, overlap_thresh, prior_for_matching,
                 bkg_label, neg_mining, neg_pos, neg_overlap, encode_target,
                 use_gpu=True):
        super(MultiBoxLoss, self).__init__()
        self.use_gpu = use_gpu
        '''有幾類'''
        self.num_classes = num_classes
        '''判定為正樣本的threshold，一般設為0.5'''
        self.threshold = overlap_thresh
        '''background自己會有一類，不用Label，假如我們有20類一樣標註0-19，下方會自己空出一類給background'''
        self.background_label = bkg_label
        self.encode_target = encode_target
        self.use_prior_for_matching = prior_for_matching
        '''OHEM，找出分得最不好的樣品，也就是confidence score比較低的正負樣品'''
        self.do_neg_mining = neg_mining
        '''負樣品與正樣品的比例，通常是3:1'''
        self.negpos_ratio = neg_pos
        self.neg_overlap = neg_overlap
        self.variance = cfg['variance']
     

    def forward(self, predictions, targets):

        '''prediction會output三個值'''
        '''loc shape: bounding box 資訊，torch.size(batch_size,num_priors,4)'''
        '''conf shape: 每一個bounding box 的信心程度，torch.size(batch_size,num_priors,num_classes)'''
        '''priors shape: 預設的defaul box， torch.size(num_priors,4)'''
        loc_data, conf_data, priors = predictions
        num = loc_data.size(0)
        priors = priors[:loc_data.size(1), :]
        num_priors = (priors.size(0))
        num_classes = self.num_classes

        # match priors (default boxes) and ground truth boxes
        loc_t = torch.Tensor(num, num_priors, 4)
        conf_t = torch.LongTensor(num, num_priors)
        for idx in range(num):
            truths = targets[idx][:, :-1].data
            labels = targets[idx][:, -1].data
            defaults = priors.data
            '''jaccard 計算每一個BBOX與ground truth的IOU'''
            match(self.threshold, truths, defaults, self.variance, labels,
                  loc_t, conf_t, idx)
        if self.use_gpu:
            loc_t = loc_t.cuda()
            conf_t = conf_t.cuda()
        '''用Variable包裝'''
        loc_t = Variable(loc_t, requires_grad=False)
        conf_t = Variable(conf_t, requires_grad=False)

        pos = conf_t > 0
        num_pos = pos.sum(dim=1, keepdim=True)


        pos_idx = pos.unsqueeze(pos.dim()).expand_as(loc_data)
        loc_p = loc_data[pos_idx].view(-1, 4)
        loc_t = loc_t[pos_idx].view(-1, 4)
        '''smooth_l1_loss 計算bounding box regression'''
        loss_l = F.smooth_l1_loss(loc_p, loc_t, size_average=False)

        # Compute max conf across batch for hard negative mining
        batch_conf = conf_data.view(-1, self.num_classes)
        loss_c = log_sum_exp(batch_conf) - batch_conf.gather(1, conf_t.view(-1, 1))

        # Hard Negative Mining
        loss_c = loss_c.view(num, -1)
        loss_c[pos] = 0
        '''排列confidence 的分數'''
        _, loss_idx = loss_c.sort(1, descending=True)
        _, idx_rank = loss_idx.sort(1)
        num_pos = pos.long().sum(1, keepdim=True)
        '''負樣品取出數量 == negpos_ratio*num_pos'''
        num_neg = torch.clamp(self.negpos_ratio*num_pos, max=pos.size(1)-1)
        neg = idx_rank < num_neg.expand_as(idx_rank)

        # Confidence Loss Including Positive and Negative Examples
        pos_idx = pos.unsqueeze(2).expand_as(conf_data)
        neg_idx = neg.unsqueeze(2).expand_as(conf_data)
        conf_p = conf_data[(pos_idx+neg_idx).gt(0)].view(-1, self.num_classes)
        targets_weighted = conf_t[(pos+neg).gt(0)]
        '''用cross_entropy做分類'''
        loss_c = F.cross_entropy(conf_p, targets_weighted, size_average=False)

        # Sum of losses: L(x,c,l,g) = (Lconf(x, c) + αLloc(x,l,g)) / N
        #double轉成torch.float64
        N = num_pos.data.sum().double()
        loss_l = loss_l.double()
        loss_c = loss_c.double()
        loss_l /= N
        loss_c /= N
        return loss_l, loss_c


## 產生我們Loss function，注意這裡的class要包含背景

In [40]:
'''要不要使用gpu'''
Use_cuda=True
criterion = MultiBoxLoss(21, 0.5, True, 0, False, 3, 0.5,False, Use_cuda,)

## 基本設定

In [41]:
ssd_net=build_ssd('train', size=300, num_classes=21)
use_pretrained=False
if use_pretrained:
    ssd_net.load_weights('./demo/ssd300_mAP_77.43_v2.pth')
net=ssd_net

In [42]:
'''要不要使用gpu'''
Use_cuda=True

'''tensor type會依照cpu或gpu有所不同'''
if torch.cuda.is_available():
    if Use_cuda:
        torch.set_default_tensor_type('torch.cuda.FloatTensor')
    else:
        print("WARNING: It looks like you have a CUDA device, but aren't " +
              "using CUDA.\nRun with --cuda for optimal training speed.")
        torch.set_default_tensor_type('torch.FloatTensor')
else:
    torch.set_default_tensor_type('torch.FloatTensor')


'''使用GPU時模型要轉成cuda'''
if Use_cuda:    
    net = net.cuda()
    
batch_size_=32
optimizer = optim.Adam(net.parameters(),lr=0.00001/batch_size_)

---

## 訓練

## 這裡我們先示範輸入的 image,Label格式，真正在訓練時，準備成一樣格式即可

In [43]:
'''輸入影像格式，假設batch size 為 4'''
image_in=torch.tensor(torch.rand(4,3,300,300),dtype=torch.float32)
'''Label格式，沒有固定長度，看圖像中有幾個label就有幾個'''
label_0=[[ 0.1804,  0.6076,  0.7701,  0.8485, 0.0000],
       [ 0.2250,  0.0000,  0.9238,  0.5641, 3.0000],
       [ 0.2250,  0.0000,  0.9238,  0.5641, 19.0000],
       [ 0.2950,  0.0000,  0.8238,  0.3641, 6.0000],]
label_1=[[ 0.1804,  0.6076,  0.7701,  0.8485, 13.0000],
       [ 0.2250,  0.0000,  0.9238,  0.5641, 11.0000],
       [ 0.2250,  0.0000,  0.9238,  0.5641, 7.0000],
       [ 0.2950,  0.0000,  0.8238,  0.3641, 5.0000],]
label_2=[[ 0.1804,  0.6076,  0.7701,  0.8485, 0.0000],
       [ 0.2250,  0.0000,  0.9238,  0.5641, 3.0000],
       [ 0.2250,  0.0000,  0.9238,  0.5641, 14.0000],
       [ 0.2950,  0.0000,  0.8238,  0.3641, 6.0000],]
label_3=[[ 0.1804,  0.6076,  0.7701,  0.8485, 0.0000],
       [ 0.2250,  0.0000,  0.9238,  0.5641, 3.0000],
       [ 0.2250,  0.0000,  0.9238,  0.5641, 19.0000],
       [ 0.2950,  0.0000,  0.8238,  0.3641, 6.0000],]

E:\using\Anaconda3\envs\dl_cv\lib\site-packages\ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


In [44]:
epochs=300
iteration=1000

In [46]:
for epoch in range(epochs):
    n=0
    loss_sum=[]
    loc_loss=[]
    conf_loss=[]
    for number__ in range(iteration) :
        '''要用Variable包裝tensor才能送入模型'''
        if Use_cuda:
            image_ = Variable(image_in.cuda())
            y = [Variable(torch.tensor(label_0).cuda(), volatile=True),Variable(torch.tensor(label_1).cuda(), 
                volatile=True),Variable(torch.tensor(label_2).cuda(), volatile=True),Variable(torch.tensor(label_3).cuda(), volatile=True)]      
        else:
            image_ = Variable(image_in)
            y = [Variable(torch.tensor(label_0), volatile=True),Variable(torch.tensor(label_1), 
                volatile=True),Variable(torch.tensor(label_2), volatile=True),Variable(torch.tensor(label_3), volatile=True)]

        '''Forward Pass'''
        out = net(image_)
        '''Regression Loss and Classification Loss'''
        loss_l,loss_c = criterion(out,y )
        loss = loss_l+ loss_c
        '''Backward'''
        loss.backward()

        loc_loss.append(loss_l.data.cpu().numpy())
        conf_loss.append(loss_c.data.cpu().numpy())
        loss_sum.append(loss.data.cpu().numpy())
        '''更新參數'''
        optimizer.step()
        '''清空Gradients'''
        optimizer.zero_grad()
        
        n+=1
        if n%10==0:
            print('BBOX Regression Loss: ', np.mean(loc_loss))
            print('Classification Loss: ', np.mean(conf_loss))
    '''儲存權重'''
    torch.save(ssd_net.state_dict(),'weights/Ｗeights.pth')

E:\using\Anaconda3\envs\dl_cv\lib\site-packages\ipykernel_launcher.py:10: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  # Remove the CWD from sys.path while we load stuff.
E:\using\Anaconda3\envs\dl_cv\lib\site-packages\ipykernel_launcher.py:11: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  # This is added back by InteractiveShellApp.init_path()


BBOX Regression Loss:  2.4812976978443286
Classification Loss:  12.130521760163484
BBOX Regression Loss:  2.4810226299144604
Classification Loss:  12.13001788104022
BBOX Regression Loss:  2.4807429937668783
Classification Loss:  12.129508388189624
BBOX Regression Loss:  2.4804586057309748
Classification Loss:  12.128992999041522
BBOX Regression Loss:  2.480168790464048
Classification Loss:  12.128471024124712
BBOX Regression Loss:  2.4798722161187063
Classification Loss:  12.12794251618562
BBOX Regression Loss:  2.4795685419960627
Classification Loss:  12.127406835808326
BBOX Regression Loss:  2.4792578767847133
Classification Loss:  12.126863663284867
BBOX Regression Loss:  2.478940045392072
Classification Loss:  12.1263136498722
BBOX Regression Loss:  2.47861477887189
Classification Loss:  12.125756609881366
BBOX Regression Loss:  2.4782820370863585
Classification Loss:  12.125191849731037
BBOX Regression Loss:  2.4779410821420176
Classification Loss:  12.124618228865256
BBOX Regress

BBOX Regression Loss:  1.8942196881329572
Classification Loss:  10.514594975224247
BBOX Regression Loss:  1.8914726893107097
Classification Loss:  10.495798633716728
BBOX Regression Loss:  1.8887009514702693
Classification Loss:  10.47697279188368
BBOX Regression Loss:  1.885770147818106
Classification Loss:  10.45840001989294
BBOX Regression Loss:  1.8827053578694661
Classification Loss:  10.439925853587964
BBOX Regression Loss:  1.8794248981240356
Classification Loss:  10.421644083658855
BBOX Regression Loss:  1.8760269770546565
Classification Loss:  10.403401983351936
BBOX Regression Loss:  1.8725877267343023
Classification Loss:  10.385068158750181
BBOX Regression Loss:  1.8691589590943891
Classification Loss:  10.366564439059285
BBOX Regression Loss:  1.865748577824345
Classification Loss:  10.347843480993202
BBOX Regression Loss:  1.8623818997983574
Classification Loss:  10.328841156108611
BBOX Regression Loss:  1.8590677167162482
Classification Loss:  10.30951192408432
BBOX Regr

BBOX Regression Loss:  0.335296360651652
Classification Loss:  3.3061601426866316
BBOX Regression Loss:  0.32870216958316756
Classification Loss:  3.2883959734881367
BBOX Regression Loss:  0.32210877029984086
Classification Loss:  3.270813122502079
BBOX Regression Loss:  0.31557653285838944
Classification Loss:  3.2533531471535007
BBOX Regression Loss:  0.3092121303817372
Classification Loss:  3.235971399001133
BBOX Regression Loss:  0.30304899165239285
Classification Loss:  3.218670795581959
BBOX Regression Loss:  0.2971178480872402
Classification Loss:  3.2014071252610945
BBOX Regression Loss:  0.29140838105001565
Classification Loss:  3.1841692481021338
BBOX Regression Loss:  0.28592280599806047
Classification Loss:  3.166949923479999
BBOX Regression Loss:  0.2806280471660473
Classification Loss:  3.1497643159294766
BBOX Regression Loss:  0.27550547137672515
Classification Loss:  3.132586250776126
BBOX Regression Loss:  0.270553567946127
Classification Loss:  3.1154377429233997
BBOX

BBOX Regression Loss:  0.004185643792152405
Classification Loss:  0.6466304072627314
BBOX Regression Loss:  0.004083877539745083
Classification Loss:  0.647024479618779
BBOX Regression Loss:  0.0040335292341532525
Classification Loss:  0.6450323340333537
BBOX Regression Loss:  0.003922574008228602
Classification Loss:  0.6428364753723145
BBOX Regression Loss:  0.003788745966222551
Classification Loss:  0.6406762539898908
BBOX Regression Loss:  0.00363896679547098
Classification Loss:  0.6387039655520592
BBOX Regression Loss:  0.003511144855508098
Classification Loss:  0.638045292808896
BBOX Regression Loss:  0.0034042145884423344
Classification Loss:  0.6356477702105486
BBOX Regression Loss:  0.0032743012954178172
Classification Loss:  0.6349371411672835
BBOX Regression Loss:  0.0031898656156327987
Classification Loss:  0.6349154677214446
BBOX Regression Loss:  0.003155805125381008
Classification Loss:  0.6345043965863058
BBOX Regression Loss:  0.0031188834826519464
Classification Loss

...

BBOX Regression Loss:  0.0007815067871922145
Classification Loss:  0.0
BBOX Regression Loss:  0.0007388807779729918
Classification Loss:  0.0
BBOX Regression Loss:  0.0007018170075887744
Classification Loss:  0.0
BBOX Regression Loss:  0.0006693742165954739
Classification Loss:  0.0
BBOX Regression Loss:  0.000640685645070065
Classification Loss:  0.0
BBOX Regression Loss:  0.0006152608892090663
Classification Loss:  0.0
BBOX Regression Loss:  0.0005924908469293128
Classification Loss:  0.0
BBOX Regression Loss:  0.0005720723463087695
Classification Loss:  0.0
BBOX Regression Loss:  0.0005536795611828817
Classification Loss:  0.0
BBOX Regression Loss:  0.0005370764998380161
Classification Loss:  0.0
BBOX Regression Loss:  0.0005219945413876831
Classification Loss:  0.0
BBOX Regression Loss:  0.0005084147835495296
Classification Loss:  0.0
BBOX Regression Loss:  0.012625952005025497
Classification Loss:  0.0
BBOX Regression Loss:  0.04289519026803955
Classification Loss:  0.0
BBOX Regre

BBOX Regression Loss:  0.03554522928987151
Classification Loss:  0.0
BBOX Regression Loss:  0.03447220067432993
Classification Loss:  0.0
BBOX Regression Loss:  0.033462370965606475
Classification Loss:  0.0
BBOX Regression Loss:  0.032510305441886427
Classification Loss:  0.0
BBOX Regression Loss:  0.03161121660176619
Classification Loss:  0.0
BBOX Regression Loss:  0.030760845773362994
Classification Loss:  0.0
BBOX Regression Loss:  0.029955335107807173
Classification Loss:  0.0
BBOX Regression Loss:  0.029191208510758692
Classification Loss:  0.0
BBOX Regression Loss:  0.028465343298825985
Classification Loss:  0.0
BBOX Regression Loss:  0.02777496257831798
Classification Loss:  0.0
BBOX Regression Loss:  0.027117565598456284
Classification Loss:  0.0
BBOX Regression Loss:  0.026490892728363084
Classification Loss:  0.0
BBOX Regression Loss:  0.026018593571307955
Classification Loss:  0.0
BBOX Regression Loss:  0.035734605517510495
Classification Loss:  0.0
BBOX Regression Loss:  0

BBOX Regression Loss:  0.023669910135678946
Classification Loss:  0.0
BBOX Regression Loss:  0.023208449561485024
Classification Loss:  0.0
BBOX Regression Loss:  0.022764794645961005
Classification Loss:  0.0
BBOX Regression Loss:  0.022337926499617817
Classification Loss:  0.0
BBOX Regression Loss:  0.021926926522559148
Classification Loss:  0.0
BBOX Regression Loss:  0.021530938666574153
Classification Loss:  0.0
BBOX Regression Loss:  0.02114915557850273
Classification Loss:  0.0
BBOX Regression Loss:  0.02078090521576255
Classification Loss:  0.0
BBOX Regression Loss:  0.020429028245179746
Classification Loss:  0.0
BBOX Regression Loss:  0.02593628991641074
Classification Loss:  0.0
BBOX Regression Loss:  0.028086884959937178
Classification Loss:  0.0
BBOX Regression Loss:  0.028274873327862524
Classification Loss:  0.0
BBOX Regression Loss:  0.028041764789329356
Classification Loss:  0.0
BBOX Regression Loss:  0.027670354458480432
Classification Loss:  0.0
BBOX Regression Loss:  

BBOX Regression Loss:  0.04567662670416808
Classification Loss:  0.0
BBOX Regression Loss:  0.045017045080400324
Classification Loss:  0.0
BBOX Regression Loss:  0.044376332726361586
Classification Loss:  0.0
BBOX Regression Loss:  0.043753724194634136
Classification Loss:  0.0
BBOX Regression Loss:  0.04314847906902691
Classification Loss:  0.0
BBOX Regression Loss:  0.04256544102748543
Classification Loss:  0.0
BBOX Regression Loss:  0.048835261240897536
Classification Loss:  0.0
BBOX Regression Loss:  0.05256565643408922
Classification Loss:  0.0
BBOX Regression Loss:  0.05316587073811726
Classification Loss:  0.0
BBOX Regression Loss:  0.05287169400138261
Classification Loss:  0.0
BBOX Regression Loss:  0.052333587670072124
Classification Loss:  0.0
BBOX Regression Loss:  0.05172066139772253
Classification Loss:  0.0
BBOX Regression Loss:  0.051093521949401796
Classification Loss:  0.0
BBOX Regression Loss:  0.05047043029837025
Classification Loss:  0.0
BBOX Regression Loss:  0.049

BBOX Regression Loss:  0.05356304998669762
Classification Loss:  0.0
BBOX Regression Loss:  0.052955974323444954
Classification Loss:  0.0
BBOX Regression Loss:  0.05236257956522357
Classification Loss:  0.0
BBOX Regression Loss:  0.051782382446113176
Classification Loss:  0.0
BBOX Regression Loss:  0.05121498685317834
Classification Loss:  0.0
BBOX Regression Loss:  0.05065995486039442
Classification Loss:  0.0
BBOX Regression Loss:  0.05011689763482622
Classification Loss:  0.0
BBOX Regression Loss:  0.049585432486971545
Classification Loss:  0.0
BBOX Regression Loss:  0.04906519020737837
Classification Loss:  0.0
BBOX Regression Loss:  0.048555838572801685
Classification Loss:  0.0
BBOX Regression Loss:  0.04805702188745839
Classification Loss:  0.0
BBOX Regression Loss:  0.04756840685121497
Classification Loss:  0.0
BBOX Regression Loss:  0.04708972636642846
Classification Loss:  0.0
BBOX Regression Loss:  0.04742389720461883
Classification Loss:  0.0
BBOX Regression Loss:  0.54214

## 想要Train VOC,COCO可以參考原Github:https://github.com/amdegroot/ssd.pytorch